<a href="https://colab.research.google.com/github/rakib06/AI/blob/master/Exercise_4_Question.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

Below is code with a link to a happy or sad dataset which contains 80 images, 40 happy and 40 sad. 
Create a convolutional neural network that trains to 100% accuracy on these images,  which cancels training upon hitting training accuracy of >.999

Hint -- it will work best with 3 convolutional layers.

In [33]:
import tensorflow as tf
import os
import zipfile


DESIRED_ACCURACY = 0.999

!wget --no-check-certificate \
    "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
    -O "/tmp/happy-or-sad.zip"

zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>DESIRED_ACCURACY):
      print('\nReached ', DESIRED_ACCURACY, '% so cancelling training!')
      self.model.stop_training = True

callbacks = myCallback()

--2020-11-09 06:16:14--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.195.128, 74.125.142.128, 74.125.20.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.195.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.01s   

2020-11-09 06:16:14 (215 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



In [52]:
# This Code Block should Define and Compile the Model
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(32, (3,3), activation='relu',),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(32, (3,3), activation='relu',),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'), # why 512 ?
  tf.keras.layers.Dense(1, activation='sigmoid') # why 1, not 10?
])

from tensorflow.keras.optimizers import RMSprop
model.compile(optimizer=RMSprop(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_26 (Conv2D)           (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 34, 34, 32)        9248      
_________________________________________________________________
max_pooling2d_28 (MaxPooling (None, 17, 17, 32)        0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 9248)            

In [53]:
# This code block should create an instance of an ImageDataGenerator called train_datagen 
# And a train_generator by calling train_datagen.flow_from_directory

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
       '/tmp/h-or-s',
       target_size=(150,150),
       batch_size=10,
       class_mode='binary')

# Expected output: 'Found 80 images belonging to 2 classes'

Found 80 images belonging to 2 classes.


In [54]:
# This code block should call model.fit and train for
# a number of epochs. 
history = model.fit(
      train_generator,
      steps_per_epoch=8,
      epochs=15,
      #verbose=1,
      callbacks=[callbacks])    
# Expected output: "Reached 99.9% accuracy so cancelling training!""

Epoch 1/15
8/8 [==============================] - 0s 16ms/step - loss: 1.0198 - accuracy: 0.4750
Epoch 2/15
8/8 [==============================] - 0s 15ms/step - loss: 0.6409 - accuracy: 0.5875
Epoch 3/15
8/8 [==============================] - 0s 16ms/step - loss: 0.4498 - accuracy: 0.8750
Epoch 4/15
8/8 [==============================] - 0s 17ms/step - loss: 0.2592 - accuracy: 0.8875
Epoch 5/15
8/8 [==============================] - 0s 18ms/step - loss: 0.1494 - accuracy: 0.9250
Epoch 6/15
8/8 [==============================] - 0s 17ms/step - loss: 0.0733 - accuracy: 0.9750
Epoch 7/15
8/8 [==============================] - 0s 17ms/step - loss: 0.0473 - accuracy: 0.9750
Epoch 8/15
8/8 [==============================] - 0s 16ms/step - loss: 0.0362 - accuracy: 0.9875
Epoch 9/15
8/8 [==============================] - 0s 17ms/step - loss: 0.0708 - accuracy: 0.9625
Epoch 10/15
7/8 [=========================>....] - ETA: 0s - loss: 0.0081 - accuracy: 1.0000
Reached  0.999 % so cancelling tra